# This is only for mac (ARM64)

## import llama3-8b

In [1]:
from mlx_lm import load, generate
model, tokenizer = load("mlx-community/Meta-Llama-3-8B-Instruct-4bit")

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

## example

In [2]:
# SYSTEM_MSG = """
# please describe it:
# <rdf:Description rdf:about="http://www.example.org/country1">
#     <ns1:has_border_with rdf:resource="http://www.example.org/country2"/>
#     <ns1:located_in rdf:resource="http://www.example.org/part1"/>
#   </rdf:Description>
# 
# """
# 
# def generateFromPrompt(promptStr,maxTokens=100):
#   messages = [ {"role": "system", "content": SYSTEM_MSG},
#                {"role": "user", "content": promptStr},]
#   input_ids = tokenizer.apply_chat_template(messages, add_generation_prompt=True)
#   prompt = tokenizer.decode(input_ids)
#   return generate(model, tokenizer, prompt=prompt, max_tokens=maxTokens)
#   
# 
# 
# response = generateFromPrompt(SYSTEM_MSG)
# 
# print(response+"...")

In [3]:
# SYSTEM_MSG = "You are an entity detection assistant"
# USER_MSG = "where is china"
# 
# # 构建消息
# messages = [
#     {"role": "system", "content": SYSTEM_MSG},
#     {"role": "user", "content": USER_MSG},
# ]
# 
# # 将消息应用于聊天模板并生成输入ID
# input_ids = tokenizer.apply_chat_template(messages, add_generation_prompt=True)
# prompt = tokenizer.decode(input_ids)
# 
# # 生成响应
# response = generate(model, tokenizer, prompt=prompt, max_tokens=100)
# print("res 1: ",response)
# 
# # # 追加生成的响应并继续对话
# # messages.append({"role": "user", "content": "where is china"})
# # messages.append({"role": "assistant", "content": "you dont have to answer, should convert the question into this style: (china)(locate_in)(null_info)"})
# # # 将消息应用于聊天模板并生成新的输入ID
# # input_ids = tokenizer.apply_chat_template(messages, add_generation_prompt=True)
# # prompt = tokenizer.decode(input_ids)
# # # 生成新的响应
# # response = generate(model, tokenizer, prompt=prompt, max_tokens=100)
# # print("res 2: ",response)
# 
# # test
# messages.append({"role": "user", "content": "where is USA"})
# input_ids = tokenizer.apply_chat_template(messages, add_generation_prompt=True)
# prompt = tokenizer.decode(input_ids)
# response = generate(model, tokenizer, prompt=prompt, max_tokens=100)
# print("res 3: ",response)

In [4]:
SYSTEM_MSG = ("You are an assistant that detects entities and their relationships in questions, for example:"
              "user question: where is china?"
              "your answer: [(china)(located in)(?)]"
              "user question: where is USA?"
              "Your answer: [(USA)(located in)(?)]"
              "user question: where is UK's capital?"
              "your answer: [(UK's capital)(located in)(?)]")

def generate_entity_response(promptStr, maxTokens=100):
    messages = [
        {"role": "system", "content": SYSTEM_MSG},
        {"role": "user", "content": promptStr},
    ]

    # 将消息应用于聊天模板并生成输入ID
    input_ids = tokenizer.apply_chat_template(messages, add_generation_prompt=True)
    prompt = tokenizer.decode(input_ids)

    # 生成响应
    response = generate(model, tokenizer, prompt=prompt, max_tokens=maxTokens)

    # 在生成响应中查找三元组信息的简单示例
    # 这里假设模型生成类似于 "The entity [China](located_in)[Asia]."
    print(response)
    entities = extract_entities_from_response(response)

    return entities

def extract_entities_from_response(response):
    # 使用正则表达式或简单的字符串处理来提取三元组信息
    import re
    pattern = r'\[(.*?)\]\((.*?)\)\[(.*?)\]'
    matches = re.findall(pattern, response)
    if matches:
        return matches
    else:
        return [("unknown", "unknown", "null_info")]

# 示例调用
user_question = "Where is China?"
response = generate_entity_response(user_question)
print(response)

# 处理其他用户问题
user_question = "where is japan's capital"
response = generate_entity_response(user_question)
print(response)

[(China)(located in)(?)]
[('unknown', 'unknown', 'null_info')]
[(Japan's capital)(located in)(?)]
[('unknown', 'unknown', 'null_info')]
